In [53]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List, Tuple, Iterable
from dataclasses import dataclass
from IPython.display import display

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.WARNING, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{
  "shell_port": 64183,
  "iopub_port": 64184,
  "stdin_port": 64185,
  "control_port": 64186,
  "hb_port": 64187,
  "ip": "127.0.0.1",
  "key": "2a1b463d-902d6f71f3707b0cf3d8c57c",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-2c47d847-0c18-4cf7-9f65-db4d1b2941df.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned ON


In [118]:
# A = np.array([
#     [0, 1, 0],
#     [0, 0, 1],
#     [0, 0, 1],
#     [1, 1, 1],
# ])

A = np.array([[0.24, 0.04, 0.72],
 [0.71, 0.20, 0.12],
 [0.36, 0.85, 0.78],])
B = A / A.sum(axis=1)[np.newaxis].T
print(A)
print(B)
print(A.sum(axis=1)[np.newaxis].T)
#np.all(A[0,:] == A, axis=1)

[[0.24 0.04 0.72]
 [0.71 0.20 0.12]
 [0.36 0.85 0.78]]
[[0.24 0.04 0.72]
 [0.69 0.19 0.12]
 [0.18 0.43 0.39]]
[[1.00]
 [1.03]
 [1.99]]


In [179]:
import numpy as np

np.set_printoptions(formatter={'float_kind': lambda x: "%.2f" % x})

class SimpleGame:
    
    
    def __init__(self, agents: int, min_opinion: float=10, max_opinion: float=20):
        self.agents = agents
        self.min_opinion = min_opinion
        self.max_opinion = max_opinion
        self.A_initial = None
        self.x_initial = None
        self.A_final = None
        self.x_final = None

    @classmethod
    def make_initial_game(cls, agents: int, opinions_range: Tuple[int, int]=(10, 20)) -> Tuple[np.ndarray, np.ndarray]:
        A = np.random.rand(agents, agents)
        A = A / A.sum(axis=1)[np.newaxis].T 
        x = np.random.randint(*opinions_range, size=(1, agents))
        return A, x.ravel()

    @classmethod
    def assign_agents_to_players(cls, agents: int, players: int=2):
        assert agents >= players
        agents_player_vector = np.random.randint(0, agents + 1, agents)
        return [
            [
                agent for agent, assigned_player in enumerate(agents_player_vector) 
                if assigned_player == player
            ]
            for player in range(1, players+1)
        ]
        
    
    @classmethod
    def check_game_solved(cls, A: np.ndarray, x: np.ndarray, accuracy: float=0.01):
        for j in range(len(x)):
            for i in range(len(x)):
                if not abs(A[i][j] - A[0][j]) < accuracy:
                    return False

        for i in range(len(x)):
            if not abs(x[i] - x[0]) < accuracy:
                return False
        return True


    def reset(self):
        self.A_initial, self.x_initial = self.make_initial_game(self.agents, 
                                                                opinions_range=(self.min_opinion, self.max_opinion))
        print('Сгенерированная игра:')
        self.print_game(self.A_initial, self.x_initial)
        
    @classmethod
    def print_game(cls, A: np.ndarray, x: np.ndarray, players: Iterable[Iterable[int]]=()):
        if A is not None:
            print(f"Матрица влияния A :\n{A}")
        print(f"Вектор мнений: \n{x}")
        for player, player_agents in enumerate(players):
            print(f"Агенты {player+1} игрока: {player_agents}")
    
    def solve(self, accuracy: float=0.001):
        self.A_final = self.A_initial.copy()
        self.x_final = self.x_initial.copy()
        iterations = 0
        while not self.check_game_solved(self.A_final, self.x_final, accuracy=accuracy):
            self.A_final = self.A_final @ self.A_final
            self.x_final = self.A_initial @ self.x_final
            iterations += 1
        return iterations
        

        
class InfluencedAgentsGame(SimpleGame):
    
    def __init__(self, agents: int, min_opinion: float=10, max_opinion: float=20, 
                 players: int=2, influenced_opinions=[1, -1]):
        assert len(influenced_opinions) == players
        super().__init__(agents, min_opinion, max_opinion)
        self.influenced_opinions = influenced_opinions
        self.players = players
        self.player_agents = []
    
    def reset(self, A=None, x=None):
        self.A_initial, self.x_initial = self.make_initial_game(self.agents, 
                                                                opinions_range=(self.min_opinion, self.max_opinion))
        if A is not None:
            self.A_initial = A
        if x is not None:
            self.x_initial = x
        self.player_agents = self.assign_agents_to_players(self.agents, self.players)
        for player, agents in enumerate(self.player_agents):
            for agent in agents:
                self.x_initial[agent] = self.influenced_opinions[player]
        print('Сгенерированная игра с купленными агентами:')
        self.print_game(self.A_initial, self.x_initial, self.player_agents)


In [182]:
simple_game = SimpleGame(agents=9, min_opinion=10, max_opinion=20)
simple_game.reset()
iterations = simple_game.solve(accuracy=0.000001)
print("-" * 30)
print(f"Решение игры с независимыми агентами [{iterations} итераций]:")
simple_game.print_game(simple_game.A_final, simple_game.x_final)


game = InfluencedAgentsGame(agents=9, min_opinion=-100, max_opinion=100, influenced_opinions=(100, -100))
game.reset(A=simple_game.A_initial, x=simple_game.x_initial)
iterations = game.solve(accuracy=0.000001)
print("-" * 30)
print(f"Решение игры с купленными агентами [{iterations} итераций]:")
game.print_game(game.A_final, game.x_final)


Сгенерированная игра:
Матрица влияния A :
[[0.05 0.14 0.15 0.11 0.15 0.05 0.04 0.25 0.06]
 [0.16 0.13 0.09 0.16 0.17 0.00 0.03 0.13 0.13]
 [0.09 0.16 0.05 0.04 0.21 0.11 0.03 0.06 0.23]
 [0.09 0.06 0.05 0.04 0.29 0.07 0.24 0.05 0.11]
 [0.10 0.14 0.10 0.16 0.13 0.11 0.13 0.03 0.10]
 [0.20 0.03 0.05 0.16 0.17 0.18 0.07 0.08 0.06]
 [0.02 0.11 0.12 0.13 0.12 0.10 0.09 0.13 0.18]
 [0.09 0.05 0.02 0.15 0.16 0.24 0.07 0.11 0.10]
 [0.02 0.10 0.15 0.12 0.11 0.17 0.11 0.05 0.17]]
Вектор мнений: 
[15 18 14 15 12 15 13 15 16]
------------------------------
Решение игры с независимыми агентами [9 итераций]:
Матрица влияния A :
[[0.09 0.10 0.09 0.12 0.17 0.11 0.10 0.09 0.12]
 [0.09 0.10 0.09 0.12 0.17 0.11 0.10 0.09 0.12]
 [0.09 0.10 0.09 0.12 0.17 0.11 0.10 0.09 0.12]
 [0.09 0.10 0.09 0.12 0.17 0.11 0.10 0.09 0.12]
 [0.09 0.10 0.09 0.12 0.17 0.11 0.10 0.09 0.12]
 [0.09 0.10 0.09 0.12 0.17 0.11 0.10 0.09 0.12]
 [0.09 0.10 0.09 0.12 0.17 0.11 0.10 0.09 0.12]
 [0.09 0.10 0.09 0.12 0.17 0.11 0.10 0.09 